# Análisis de Datos Ambientales por Estación y Franja Horaria

### 1. Rutas de archivos

Se definen rutas robustas para cargar y guardar los archivos de datos, permitiendo ejecutar el notebook o scripts desde diferentes ubicaciones sin errores de path.

In [ ]:
from pathlib import Path
import os

def locate_data_processed(filename='datos_sima_limpios_combined.csv'):
    try:
        start = Path(__file__).resolve().parent.parent
    except NameError:
        start = Path.cwd()

    candidates = [start, Path.cwd(), start.parent, Path.cwd().parent]
    for c in candidates:
        if c is None:
            continue
        for p in [c] + list(c.parents)[:5]:
            dp = p / 'data_processed'
            if dp.exists() and (dp / filename).exists():
                return dp

    for c in candidates:
        dp = Path(c) / 'data_processed'
        if dp.exists():
            return dp

    return Path(start) / 'data_processed'

DATA_DIR = locate_data_processed()
INPUT_CSV = DATA_DIR / 'datos_sima_limpios_combined.csv'
OUT_AGG = DATA_DIR / 'datos_aggregados_dia_estacion_franja.csv'

print('DATA_DIR resolved to:', DATA_DIR)
print('INPUT_CSV exists?:', INPUT_CSV.exists())

DATA_DIR resolved to: c:\Users\anala\Documents\Semestre 5\Reto-multivariados-equipo-5\data_processed
INPUT_CSV exists?: True


In [ ]:
# Escribe un csv y devuelve la ruta como str
import pandas as pd

def format_calendar_for_csv(calendario, out_path):
    rows = []
    for est, intervals in calendario.items():
        for s, e in intervals:
            rows.append({'estacion': est, 'start': s, 'end': e})
    df_cal = pd.DataFrame(rows)
    out = Path(out_path)
    out.parent.mkdir(parents=True, exist_ok=True)
    df_cal.to_csv(out, index=False)
    return str(out)

### 2. Calendario por estación

Se especifican los periodos escolares para cada estación de monitoreo. Estos periodos se usan para identificar si una fecha corresponde a días de clase (útil para análisis de exposición en población escolar).

In [60]:
CALENDARIO_POR_ESTACION = {
    'CENTRO': [
        ('2023-01-09 00:00:00', '2023-05-12 00:00:00'),
        ('2023-08-07 00:00:00', '2023-12-01 00:00:00'),
        ('2024-01-15 00:00:00', '2024-05-10 00:00:00'),
        ('2024-08-05 00:00:00', '2024-11-29 00:00:00'),
        ('2025-01-06 00:00:00', '2025-05-09 00:00:00')
    ],
    'SUROESTE2': [
        ('2023-01-09 00:00:00', '2023-05-12 00:00:00'),
        ('2023-08-07 00:00:00', '2023-12-06 00:00:00'),
        ('2024-01-10 00:00:00', '2024-05-24 00:00:00'),
        ('2024-08-05 00:00:00', '2024-12-05 00:00:00'),
        ('2025-01-13 00:00:00', '2025-05-23 00:00:00')
    ],
    'SUR': [
        ('2023-01-23 00:00:00', '2023-06-30 00:00:00'),
        ('2023-08-07 00:00:00', '2023-12-06 00:00:00'),
        ('2024-01-22 00:00:00', '2024-06-28 00:00:00'),
        ('2024-08-05 00:00:00', '2024-12-05 00:00:00'),
        ('2025-01-20 00:00:00', '2025-06-27 00:00:00')
    ],
    'NORTE2': [
        ('2023-01-09 00:00:00', '2023-07-01 00:00:00'),
        ('2023-08-23 00:00:00', '2023-12-15 00:00:00'),
        ('2024-01-08 00:00:00', '2024-07-16 00:00:00'),
        ('2024-08-26 00:00:00', '2024-12-18 00:00:00'),
        ('2025-01-09 00:00:00', '2025-07-16 00:00:00')
    ]
}

In [43]:
def load_data(path=INPUT_CSV):
    if not path.exists():
        raise FileNotFoundError(f"Archivo no encontrado: {path}. Ejecute la etapa 1 primero.")
    df = pd.read_csv(path, parse_dates=['date'], dayfirst=False)
    return df

In [44]:
def create_time_vars(df):
    df = df.copy()
    if 'date' not in df.columns:
        raise KeyError('La columna `date` no está presente.')
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date'])
    df['anio'] = df['date'].dt.year
    df['mes'] = df['date'].dt.month
    df['dia'] = df['date'].dt.day
    df['hora'] = df['date'].dt.hour

    try:
        df['dia_semana'] = df['date'].dt.day_name(locale='es_ES')
    except Exception:
        df['dia_semana'] = df['date'].dt.day_name()
    return df

In [ ]:
def assign_franja(h):
    " hora en entero"
    if h in [7, 8]:
        return 'pico_7_9'
    elif h in [10, 11]:
        return 'ref_10_12'
    else:
        return 'fuera_franja'

### 3. Creación de variables temporales y de franja horaria

Se generan variables auxiliares a partir de la fecha: año, mes, día, hora y día de la semana. Además, se asigna la franja horaria relevante para el análisis (pico, referencia, fuera de franja) según la hora.

In [ ]:
def assign_clase_from_calendar(df, calendario=CALENDARIO_POR_ESTACION):
    df = df.copy()
    df['clase'] = 0
    # Preprocesar calendario, convertir a datetimes
    cal_proc = {}
    for est, intervals in calendario.items():
        parsed = []
        for start_s, end_s in intervals:
            try:
                s = pd.to_datetime(start_s)
                e = pd.to_datetime(end_s)
                parsed.append((s, e))
            except Exception:
                continue
        if parsed:
            cal_proc[est] = parsed

    missing_stations = set(df['estacion'].dropna().unique()) - set(cal_proc.keys())
    if missing_stations:
        print(f"Advertencia: las siguientes estaciones no están en el calendario y recibirán clase=0: {sorted(list(missing_stations))}")

    def is_in_class(row):
        est = row['estacion']
        dt = row['date']
        if pd.isna(est) or pd.isna(dt):
            return 0
        # If station not in calendar, it's outside class
        if est not in cal_proc:
            return 0
        # Sólo lunes-viernes
        if dt.weekday() >= 5:
            return 0
        for (s, e) in cal_proc[est]:
            # comparar solo fecha (sin hora)
            if s.normalize() <= dt.normalize() <= e.normalize():
                return 1
        return 0

    df['clase'] = df.apply(is_in_class, axis=1)
    return df

### 4. Asignación de la variable `clase`

Se utiliza el calendario escolar para cada estación y se asigna la variable `clase`:
- `1` si la fecha corresponde a un día entre semana dentro de un periodo escolar para la estación.
- `0` en caso contrario.

Esto permite distinguir registros asociados a periodos de clases escolares.

In [ ]:
def filter_and_aggregate(df):
    "Selecciona las horas establecidad y asigna si pertenece o no"
    df = df.copy()
    # Mantener sólo parámetros esperados
    expected = ['NO2', 'CO', 'PM2.5', 'PM10', 'O3']
    df = df[df['parametro'].isin(expected)]

    # Filtrar a las horas exactas solicitadas
    target_hours = [7, 8, 10, 11]
    df = df[df['hora'].isin(target_hours)]

    # Asignar franja según hora
    df['franja_horaria'] = df['hora'].apply(assign_franja)

    # Seleccionar y devolver columnas deseadas
    cols = ['date','anio','mes','dia','hora','estacion','franja_horaria','parametro','valor','clase']
    # Asegurar que las columnas existan en el df
    existing = [c for c in cols if c in df.columns]
    out = df[existing].copy()

    # Orden
    final_cols = [c for c in cols if c in out.columns]
    return out[final_cols]

### 5. Filtrado y exportación de datos agregados

Se seleccionan únicamente los registros de las horas 7, 8, 10 y 11, y se asigna la franja horaria correspondiente. El dataset final incluye las variables clave para análisis posteriores y se exporta a un archivo CSV para su uso en modelos o visualizaciones.

In [49]:
def save_aggregated(agg_df, path=OUT_AGG):
    path.parent.mkdir(parents=True, exist_ok=True)
    agg_df.to_csv(path, index=False)
    print(f"✓ Agregados guardados en: {path}")

In [50]:
def compute_acf(series, nlags=30):
    if acf is None:
        raise ImportError('statsmodels no disponible. Instale statsmodels para usar ACF.')
    series = series.dropna()
    return acf(series, nlags=nlags, fft=False)

In [51]:
def compute_ljungbox(series, lags=[10]):
    if acorr_ljungbox is None:
        raise ImportError('statsmodels no disponible. Instale statsmodels para Ljung-Box.')
    series = series.dropna()
    res = acorr_ljungbox(series, lags=lags, return_df=True)
    return res

In [52]:
def compute_durbin_watson(residuals):
    if durbin_watson is None:
        raise ImportError('statsmodels no disponible. Instale statsmodels para Durbin-Watson.')
    return durbin_watson(residuals)

In [ ]:
def main():
    print('\nParte 2: creación de variables y agregación')
    print(f'Leyendo: {INPUT_CSV}')
    df = load_data(INPUT_CSV)
    print(f'Registros cargados: {len(df):,}')

    df = create_time_vars(df)
    print('Variables temporales creadas: anio, mes, dia, hora, dia_semana')

    print('Asignando variable `clase` según calendario...')
    if not CALENDARIO_POR_ESTACION:
        print('  ATENCIÓN: `CALENDARIO_POR_ESTACION` está vacío. Edite el script y agregue los periodos por estación antes de ejecutar para obtener `clase=1` cuando corresponda.')
    # Exportar versión normalizada
    try:
        cal_export_path = DATA_DIR / 'calendario_estaciones_normalized.csv'
        norm_out = format_calendar_for_csv(CALENDARIO_POR_ESTACION, cal_export_path)
        if norm_out:
            print(f'  Calendario normalizado guardado en: {norm_out}')
    except Exception as e:
        print(f'  No fue posible exportar calendario normalizado: {e}')

    df = assign_clase_from_calendar(df, calendario=CALENDARIO_POR_ESTACION)

    print('Filtrando franjas y agregando por día-estación-franja...')
    agg = filter_and_aggregate(df)
    save_aggregated(agg)

    print('\nTareas siguientes sugeridas:')
    print('- Rellenar `CALENDARIO_POR_ESTACION` con los intervalos reales por estación.')
    print('- Ejecutar nuevamente para obtener `clase` correcta y luego aplicar pruebas ACF/Ljung-Box/Durbin-Watson con las funciones disponibles.')

In [74]:
main()


== Parte 2: creación de variables y agregación ==
Leyendo: c:\Users\anala\Documents\Semestre 5\Reto-multivariados-equipo-5\data_processed\datos_sima_limpios_combined.csv
Registros cargados: 1,804,203
Registros cargados: 1,804,203
Variables temporales creadas: anio, mes, dia, hora, dia_semana
Asignando variable `clase` según calendario...
  DEBUG: DATA_DIR=c:\Users\anala\Documents\Semestre 5\Reto-multivariados-equipo-5\data_processed
  DEBUG: format_calendar_for_csv defined? True
  DEBUG: cal_export_path=c:\Users\anala\Documents\Semestre 5\Reto-multivariados-equipo-5\data_processed\calendario_estaciones_normalized.csv
  Calendario normalizado guardado en: c:\Users\anala\Documents\Semestre 5\Reto-multivariados-equipo-5\data_processed\calendario_estaciones_normalized.csv
Variables temporales creadas: anio, mes, dia, hora, dia_semana
Asignando variable `clase` según calendario...
  DEBUG: DATA_DIR=c:\Users\anala\Documents\Semestre 5\Reto-multivariados-equipo-5\data_processed
  DEBUG: form